In [5]:
import pandas as pd 
import os 

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import seaborn as sns

from datasets import load_dataset
import numpy as np
import json

In [2]:
pwd

'/home/al2644/research/codebase/reasoning/perturb-r/notebook'

In [4]:
cd /home/al2644/research/codebase/reasoning/perturb-r

/home/al2644/research/codebase/reasoning/perturb-r


# Reasoning Results

In [80]:
root = "./results/aime2425/benchmark"
k = 8
for fname in os.listdir(root):
    if 'correct' not in fname:
        print(fname)
        df = pd.read_pickle(os.path.join(root, fname))
        pass_at_k = df.groupby("problem")[["correct"]].max()["correct"].mean()
        print(f"Pass@{k} Accuracy: {pass_at_k}")

Qwen3-1.7B.pickle
Pass@8 Accuracy: 0.6166666666666667
Qwen3-1.7B_nothinking.pickle
Pass@8 Accuracy: 0.2833333333333333
Qwen3-8B.pickle
Pass@8 Accuracy: 0.75
Qwen3-8B_nothinking.pickle
Pass@8 Accuracy: 0.4166666666666667


In [86]:
df = pd.read_pickle(os.path.join(root, "Qwen3-8B_nothinking.pickle"))

In [97]:
df

,problem,solution,source,response,correct
0,"Let $x,y$ and $z$ be positive real numbers tha...",33,aime 24,We are given a system of logarithmic equations...,1.0
1,"Let $x,y$ and $z$ be positive real numbers tha...",33,aime 24,We are given the system of equations:\n\n$$\n\...,1.0
2,"Let $x,y$ and $z$ be positive real numbers tha...",33,aime 24,We are given three logarithmic equations invol...,0.0
3,"Let $x,y$ and $z$ be positive real numbers tha...",33,aime 24,We are given the following system of equations...,1.0
4,"Let $x,y$ and $z$ be positive real numbers tha...",33,aime 24,We are given the following system of logarithm...,1.0
...,...,...,...,...,...
475,There are exactly three positive real numbers ...,240,aime 25,We are given the function:\n\n$$\nf(x) = \frac...,0.0
476,There are exactly three positive real numbers ...,240,aime 25,We are given the function:\n\n$$\nf(x) = \frac...,0.0
477,There are exactly three positive real numbers ...,240,aime 25,We are given the function:\n\n$$\nf(x) = \frac...,0.0
478,There are exactly three positive real numbers ...,240,aime 25,We are given the function:\n\n$$\nf(x) = \frac...,0.0


In [39]:
reasoning, answer = df["response"][0].split("</think>")

In [63]:
from utils.process_thinking import chunk as chunk_tool
from reward_score.math import last_boxed_only_string, remove_boxed

In [69]:
chunks = chunk_tool(reasoning,30)
final_answer = remove_boxed(last_boxed_only_string(reasoning))

prompt = ""

for idx, chunk in enumerate(chunks):
    if final_answer not in chunk:
        continue
    prompt += f"###START OF CHUNK {idx}\n"
    prompt += chunk
    prompt += f"\n###END OF CHUNK {idx}\n"

In [70]:
instruction = """<instruction>
You are provided a chain of chunked model reasoning trace of deriving a solution to a problem. Your job is to identify where the answer is first reached in the reasoning chain and report the corresponding chunk number. The model may later verify or try alternative approach, but you should only focus on where the answer is first derived.

You MUST include the chunk number inside <answer_chunk></answer_chunk>. This is a hard requirement for later processing.
</instruction>\n\n
The final answer is: 1
"""
input_str = instruction + prompt

In [72]:
print(input_str)

<instruction>
You are provided a chain of chunked model reasoning trace of deriving a solution to a problem. Your job is to identify where the answer is first reached in the reasoning chain and report the corresponding chunk number. The model may later verify or try alternative approach, but you should only focus on where the answer is first derived.

You MUST include the chunk number inside <answer_chunk></answer_chunk>. This is a hard requirement for later processing.
</instruction>


The final answer is: 1
###START OF CHUNK 0
Okay, so I have this problem where I need to compute the asymptotic relative efficiency (ARE) for estimating \( e^{-\lambda} \) in a Poisson process. The expression given is:

\[
\text{ARE} = \left[\frac{e^{-\lambda}}{\left(\frac{n-1}{n}\right)^{n\lambda}\log\left( \frac{n-1}{n}\right)^n}\right]^2
\]
###END OF CHUNK 0
###START OF CHUNK 3
The numerator is \( e^{-\lambda} \). The denominator has two main parts: \( \left(\frac{n-1}{n}\right)^{n\lambda} \) and \( \